In [2]:
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import json
import argparse
import matplotlib
import matplotlib.pyplot as plt
import os
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

apiKey = 'X5R4ATFGQ5WSGI6K'

In [3]:
## ts = TimeSeries(key=apiKey, output_format='pandas')
## data,meta_data = ts.get_intraday(symbol='TSLA',interval='1min',outputsize='full')
## data.describe()


In [4]:
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import time
from datetime import datetime
import os
import json

# ####################################################
# will use this to prevent going over API call limit
stringptime = datetime.now()
current_date = stringptime.strftime('%m-%d-%Y')
count = 0


def check_count():
    global count, current_date
    if os.path.isfile(f'{current_date}.txt'):
        with open(f'{current_date}.txt', 'r') as f:
            temp = f.readline()
            temp.replace("'", "")  # a bit of cleaning to be sure
            temp.strip()  # a bit of cleaning to be sure
            count = int(temp)
            count += 1
            # print(count)
        with open(f'{current_date}.txt', 'w') as f:
            f.write(str(count))
            # print(count)

    else:
        with open(f'{current_date}.txt', 'w') as f:
            f.write(str(count))
            # print(count)
    return count

# ####################################################


def save_dataset(symbol, time_window):
    credentials = json.load(open('creds.json', 'r'))
    api_key = credentials['av_api_key']
    # print(symbol, time_window)
    ts = TimeSeries(key=api_key, output_format='pandas')
    if time_window == 'intraday':
        data, meta_data = ts.get_intraday(
            symbol = symbol, interval='1min', outputsize='full')
    elif time_window == 'daily':
        data, meta_data = ts.get_daily(symbol, outputsize='full')
    elif time_window == 'daily_adj':
        data, meta_data = ts.get_daily_adjusted(symbol, outputsize='full')

    print(symbol, 'info saved as ' + f'{symbol}_{time_window}.csv')
    print('*' * 58)
    pprint(data.head(1))
    print('-' * 58)
    pprint(data.tail(1))
    print('*' * 58)
    print()
    data.to_csv(f'./Stockcsvs/{symbol}_{time_window}.csv')


if __name__ == "__main__":

    if count <= 500:  # API limit of 500 calls per day
        # get list of stock symbols from file and strip newline
        with open('Stock symbols.txt', 'r') as f:
            symbol_list = [i.replace('\n', '') for i in f]
    
        for symbols in symbol_list:
            if check_count() <= 500:
                print('*' * 58)
                print('API calls today:', count)
                print('Getting data for stock ' + symbols)
                save_dataset(symbols, 'daily')  # daily, intraday, or daily_adj
                time.sleep(13)  # limited to 5 API requests per minute max 500 per day
            else:
                print('You have exceeded you max API calls for the day')
                print('Try again tomorrow.')



**********************************************************
API calls today: 0
Getting data for stock MSFT
MSFT info saved as MSFT_daily.csv
**********************************************************
            1. open  2. high  3. low  4. close   5. volume
date                                                      
2021-01-28   235.61   242.64  235.09    238.93  49111159.0
----------------------------------------------------------
            1. open  2. high  3. low  4. close   5. volume
date                                                      
1999-11-01    93.25    94.19   92.12     92.37  26630600.0
**********************************************************

**********************************************************
API calls today: 1
Getting data for stock GOOGL
GOOGL info saved as GOOGL_daily.csv
**********************************************************
            1. open  2. high  3. low  4. close  5. volume
date                                                     
2021-01-28